In [2]:
from db import db
import pymongo
import time

one = next(db.repka_logs.aggregate([
    {'$sort': { 'time': -1 }},
    { '$match': { 'is_viewed': { "$ne": True }, 'app': { "$eq": "hagi" } } },
    {'$skip': 0 },
    {'$limit': 1 },
]))




text = []

for one in db.repka_logs.find({ "session_id": one['session_id'] }).sort([("message_id", 1)]):
    text.append(f"> {one['request']}\n< {one['response']}\n")

print("\n".join(text))



db.repka_logs.update_many({'session_id': {'$eq': one["session_id"]}}, {"$set": { 'is_viewed': True }} )

> 
< Хаги - кукла из игры.
 Ожила так что беги.
 Я ХАГИ ВАГИ. Я буду 

> что сказать ты повторяй за мной
< что сказать ты повторяй за мной

> сколько тебе лет
< сколько тебе лет

> помощь
< Они научили меня играть в повторюшу. Я буду играть с тобой. Я по

> я боюсь
< я боюсь

> все я выхожу из игры
< все я выхожу из игры

> выход
< На этот раз тебе удалось убежать. Но мы ещё поиграем.



In [3]:
# db.command({ 'collStats' : "repka_logs", 'scale': 1024*1024 })

In [6]:
for one in db.repka_logs.find({ "app": 'hagi' }).sort([("message_id", 1)]):
    if one['request']: print(f"> {one['request']}")

> да
> аша
> кодовое слово
> кодовое слов
> кодовое слов 2
> давай поиграем
> как у тебя дела
> выход
> выход
> выход
> выход
> помощь
> я сплю всегда
> она сейчас не сработает
> привет
> я ничего не поняла
> кто ты
> сашка
> что сказать ты повторяй за мной
> кодовое слов 3
> помощь
> помощь
> что ты умеешь
> не повторюшка
> тебе нужно быстрее говори что нибудь
> я хаги ваги
> помощь
> она повторяет
> ты хаги ваги
> сколько тебе лет
> выход
> выход
> кукла из игры ожила так что беги
> ева еще раз повтори то что ты сказала
> где ваги а где ты родился
> что ты умеешь
> слишком много да
> а я кто такой
> помощь
> что ты умеешь
> привет
> где прячешся
> понятно а ты кто такой ты говоришь
> а я кто такой отвечай другу если миссия я хаги ваги
> я боюсь
> выход страшно там нет картинок ничего нет
> хаги ваги сколько тебе лет
> ты где
> ты
> нет это я
> все я выхожу из игры
> а где магия что ты умеешь
> ты здесь
> ты
> не понимает ничего
> выход
> ты
> ты не смейся а я хаги ваги
> я не понимаю